In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

2023-07-07 10:26:01.048929: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 10:26:01.220446: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-07 10:26:02.101370: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-07 10:26:02.101444: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [11]:
loader = LJDataLoader(data_path=file_path, filter_cols=['ElevenLabsCloneClip'])
loader.splitData()

source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

source_architectures = ['ElevenLabs', 'UberDuck']
new_col_name = 'EL_UD_Fake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

source_architectures = ['RandWaveFake', 'ElevenLabs', 'UberDuck']
new_col_name = 'Fake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

In [3]:
def generate_split(fake_cols, file_path):

    
    loader = LJDataLoader(data_path=file_path, filter_cols=['ElevenLabsCloneClip'])
    loader.splitData()

    source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
    new_col_name = 'RandWaveFake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['ElevenLabs', 'UberDuck']
    new_col_name = 'EL_UD_Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['RandWaveFake', 'ElevenLabs', 'UberDuck']
    new_col_name = 'Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

    #data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
    data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=fake_cols)
    
    train_count = data_df[data_df['type'] == 'train'].shape[0]
    dev_count = data_df[data_df['type'] == 'dev'].shape[0]
    test_count = data_df[data_df['type'] == 'test'].shape[0]

    print(f'# of Train instances: {train_count}')
    print(f'# of Dev instances: {dev_count}')
    print(f'# of Test instances: {test_count}')
    
    return data_df

In [4]:
df = generate_split(['ElevenLabs', 'UberDuck', 'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow'], '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv')
df

# of Train instances: 78330
# of Dev instances: 26110
# of Test instances: 26110


,type,id,architecture,path,label,multiclass_label
0,train,LJ034-0127,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ024-0083,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ041-0054,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ038-0025,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ011-0210,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
...,...,...,...,...,...,...
130545,test,LJ013-0236,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130546,test,LJ028-0195,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130547,test,LJ012-0142,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
130548,test,LJ037-0038,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9


In [5]:
df[['architecture', 'label']].value_counts()

architecture       label
ElevenLabs         1        13055
Full_Band_MelGan   1        13055
HifiGan            1        13055
MelGan             1        13055
MelGanLarge        1        13055
Multi_Band_MelGan  1        13055
Parallel_WaveGan   1        13055
Real               0        13055
UberDuck           1        13055
Waveglow           1        13055
dtype: int64

In [6]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [7]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [8]:
def run(fake_cols, metadata_path, name, data_df=None):
    if data_df is None:
        data_df = generate_split(fake_cols, metadata_path)
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    results.to_csv(f'/home/ubuntu/data/results/7-7-23/{name}.csv', index=False)
    return results
    
    
    

In [9]:
agg_df = pd.DataFrame(columns=['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'laundered', 'eer_score', 'eer_threshold'])

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['ElevenLabs'], file_path, '16KHz_ElevenLabs')
results['laundered'] = 0
agg_df = agg_df.append(results)

In [ ]:
## SB_Comment - where are the final uned cadence results? I can see the params used below but dont think they're being used 
## SB_Comment - should set the window size and threshold as global params e.g. uberduck_window_size, elevenlabs_window_size, general_window_size etc 

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['UberDuck'], file_path, '16KHz_UberDuck')
results['laundered'] = 0
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['RandWaveFake'], file_path, '16KHz_RandWaveFake_B')
results['laundered'] = 0
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck')
results['laundered'] = 0
agg_df = agg_df.append(results)


In [10]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix')
results['laundered'] = 0
agg_df = agg_df.append(results)


# of Train instances: 31332
# of Dev instances: 10444
# of Test instances: 10444
[NeMo I 2023-07-07 10:26:07 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-07-07 10:26:07 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-07-07 10:26:07 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-07 10:26:08 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-07-07 10:26:08 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-07-07 10:26:08 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-07-07 10:26:08 features:267] PADDING: 16
[NeMo I 2023-07-07 10:26:09 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 4/4 [01:12<00:00, 18.18s/it]


Initializing data...
Standardizing features...


KeyboardInterrupt: 

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['EL_UD_Fake'], file_path, '16KHz_EL_UD_Fake_B')
results['laundered'] = 0
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['Fake'], file_path, '16KHz_Fake')
results['laundered'] = 0
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['ElevenLabs'], file_path, '16KHz_ElevenLabs_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['UberDuck'], file_path, '16KHz_UberDuck_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['RandWaveFake'], file_path, '16KHz_RandWaveFake_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['EL_UD_Fake'], file_path, '16KHz_EL_UD_Fake_Laundered_B')
results['laundered'] = 1
agg_df = agg_df.append(results)


In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['Fake'], file_path, '16KHz_Fake_Laundered_B')
results['laundered'] = 1
agg_df = agg_df.append(results)


In [ ]:
agg_df.to_csv('/home/ubuntu/data/results/6-30-23/agg_results.csv', index=False)

In [ ]:
import os
directory = '/home/ubuntu/data/TIMIT converted/wav_files'
file_count = len(list(os.listdir(directory)))
file_count